Tutorial 4: Integrating Multimodal Data with Ternary Plots
==========================================================

In Tutorial 2, we showed how to use the Laplacian score to determine whether a cell feature varies throughout the morphology space. Here, we discuss another technique to relate morphological distances to other cell features; in particular, we relate morphological distances to electrophysiological and molecular features of the cells. This technique allows us to identify variation in cells which is reflected _well_ through at least one of the indicators, but is reflected _poorly_ in the other indicators. It thus reveals "blind spots" in one of the measurement techniques, demonstrating the failure of one of the modes to capture distinctions which are well-reflected in the other modes.

Suppose that we have a set of cells $c_1,\dots, c_n$, and we associate to each cell an electrophysiological feature vector $e_1,\dots, e_n\in\mathbb{R}^k$, and a molecular feature vector $m_1,\dots, m_n\in \mathbb{R}^s$. Both the sets $\{e_1,\dots, e_n\}$ and $\{m_1,\dots, m_n\}$ can be regarded as spaces, and we can interpret the distance $\lVert e_i - e_j\rVert$ as the difference between cells $c_i,c_j$ which is visible from the point of view of electrophysiology; likewise for the molecular feature vectors. Together with the Gromov-Wasserstein space of morphological distances, this gives three different manifestations of the cell space, where the notion of distance of each space reflects one aspect of the cell.

Speaking broadly, we expect cells that are very different morphologically and very different electrophysiologically to be very different in their gene expression, as well. If there are many cell pairs in the data where large morphological and large electrophysiological distances aren't accompanied by large transcriptomic differences, it raises the question of why there is substantial variation in one variable not reflected in the others. For example, we might ask whether the transcriptomic methods are sufficiently comprehensive. This leads us to plot morphological, electrophysiological, and transcriptomic distances together.

This tutorial follows the dataset from the paper [Phenotypic variation of transcriptomic cell types in mouse motor cortexl](https://www.nature.com/articles/s41586-020-2907-3) by Scala et. al.
This [folder](https://www.dropbox.com/scl/fo/qqvbjfhitcaetxf4u6i87/h?rlkey=rn5xpegeden8sdmirjl1fav2j&dl=0) contains the SWC files and metadata we will use for the analysis; we refer to it below as `bd` for "base directory".

In [1]:
import os
import scipy
from scipy.spatial.distance import squareform, pdist
from os.path import join
import pandas as pd
from cajal.utilities import dist_mat_of_dict, read_gw_dists
from cajal.ternary import ternary_distance

import numpy as np
from cajal import run_gw, sample_swc, swc

bd = '/home/patn/dropbox/PGC012_Gromov_Wasserstein/Analyses/PGC012.a28/'

As in the other tutorials, we compute the intracell distance matrices and then the Gromov-Wasserstein distance matrix.

In [2]:
sample_swc.compute_icdm_all_geodesic(
    infolder=join(bd,'swc'),
    out_csv=join(bd,'geodesic_100_icdm.csv'),
    n_sample=100,
    num_processes=20,
    preprocess=swc.preprocessor_geo([1,3,4])
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 644/645 [01:31<00:00,  7.06it/s]


[]

In [3]:
run_gw.compute_gw_distance_matrix(
    intracell_csv_loc=join(bd,'geodesic_100_icdm.csv'),
    gw_dist_csv_loc=join(bd,'geodesic_100_gw.csv'),
    num_processes=20,
    verbose=True
)

  0%|          | 0/207690 [00:00<?, ?it/s]

(array([[ 0.        , 41.50872827, 30.44856915, ..., 36.52807527,
         35.49565752, 33.33158221],
        [41.50872827,  0.        , 36.38626911, ..., 30.27160567,
         36.05393378, 25.02308163],
        [30.44856915, 36.38626911,  0.        , ..., 30.49769085,
         30.84067168, 35.80320063],
        ...,
        [36.52807527, 30.27160567, 30.49769085, ...,  0.        ,
         34.72089971, 23.26927649],
        [35.49565752, 36.05393378, 30.84067168, ..., 34.72089971,
          0.        , 35.45859098],
        [33.33158221, 25.02308163, 35.80320063, ..., 23.26927649,
         35.45859098,  0.        ]]),
 None)

The file "m1_patchseq_meta_data.csv" contains the RNA families for the samples profiled in the study.

In [4]:
metadata = pd.read_csv(join(bd,"m1_patchseq_meta_data.csv"), delimiter="\t", index_col=1)
# Start with 1329 rows × 32 columns

cells, gw_dist_dict = read_gw_dists(join(bd,'geodesic_100_gw.csv'),header=True)

metadata = metadata.loc[cells]
metadata = metadata.loc[metadata['RNA family']!='low quality']

# Labels for the classifier
print(metadata['RNA family'].unique())

['ET' 'IT' 'Pvalb' 'Vip' 'Sst' 'Lamp5' 'CT' 'Sncg' 'NP']


The files containing electrophysiological data and the exon counts have been cleaned to discard samples containing NaN values.

In [5]:
ephys_data = pd.read_csv(join(bd,'ephys_data.csv'),index_col='cell id')
exon_data = pd.read_csv(join(bd,'exon_data.csv'), index_col='cell id')

We restrict to the samples for which we have all threee - morphological data, exon counts, and electrophysiology data.

In [6]:
common_cells = list(set(metadata.index).intersection(set(exon_data.index)).intersection(ephys_data.index))
metadata = metadata.loc[common_cells]
exon_data = exon_data.loc[common_cells]
ephys_data = ephys_data.loc[common_cells]

We normalize the columns in the electrophysiology data so that they have mean 0 and standard deviation 1.
The "distance" in the electrophysiology space between two samples is defined to be 1 minus the Pearson correlation coefficient between the data for the samples. This is not a metric, but it is a common technique.

In [7]:
ephys_data = (ephys_data - ephys_data.mean())
ephys_data /= ephys_data.std()
ephys_dmat = (1-ephys_data.transpose().corr())

The following heuristic transformation is meant to narrow the long tails at the end of the distribution of exon counts.
As with the electrophysiology data, we use "correlation distance" as our notion of dissimilarity between samples.

In [ ]:
exon_data = np.log(5000 * exon_data + 1)
exon_dmat = (1-((exon_data.transpose()).corr()))

In [ ]:
def ternary_distance_RNA(RNA_family, gw_dist_dict, ephys_data, exon_dmat, metadata):
    cell_ids = metadata.index[metadata['RNA family']== RNA_family]
    exon_dmat_local = exon_dmat.loc[cell_ids][cell_ids]
    ephys_dmat_local = ephys_dmat.loc[cell_ids][cell_ids]
    gw_dmat_local = dist_mat_of_dict(gw_dist_dict,cell_ids)
    ternary_distance(gw_dmat_local, 'GW', ephys_dmat_local, 'ephys', exon_dmat_local, 'exon', density_estimation = 'gaussian_kde', bins=5, contour_lines=4,alpha=.4, s=4)

In [ ]:
large_families = [ family for family in metadata['RNA family'].unique() if (metadata['RNA family'] == family).sum() >= 10 ]
RNA_family = large_families[0] # Sst
print(RNA_family)

In [ ]:
ternary_distance_RNA(large_families[0], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[1], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[2], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[3], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[4], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[5], gw_dist_dict, ephys_dmat, exon_dmat, metadata)

In [ ]:
ternary_distance_RNA(large_families[6], gw_dist_dict, ephys_dmat, exon_dmat, metadata)